as discussed on the discord:
*   `data_grouped` is the dataframe in format 2 (cumulative)
*   `data_grouped_country` is the dataframe in format 1. three new columns named `[columnName]_diff` have been added to this dataframe, representing the **non-cumulative** count of the appropriate `[columnName]` column. **[columnName] belongs to {'Confirmed', 'Deaths', 'Recovered'}**

To use this code, grab the appropriate data files from the link below, and put them in a folder named `novel-corona-virus-2019-dataset`.
https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset/data

### import relevant python libraries

In [1]:
# !pip install geojson

In [2]:
%matplotlib inline
import numpy as np
import altair as alt
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
#import geojson
from pprint import pprint
import json
from pathlib import Path
#import seaborn as sns
import requests

In [3]:
def fontforchart():
    font = 'Calibri'
    
    return {
        "config" : {
             "title": {'font': font,
                       'fontSize' : 20},
             "axis": {
                  "labelFont": font,
                  "titleFont": font,
                  "fontSize" : 16
             },
            "header": {
                  "labelFont": font,
                  "titleFont": font,
                  "fontSize" : 16
             },
             "legend": {
                  "labelFont": font,
                  "titleFont": font,
                  "fontSize" : 16
             }
        }
    }

alt.themes.register('font', fontforchart)
alt.themes.enable('font')

ThemeRegistry.enable('font')

In [4]:
#alt.renderers.enable('notebook')

In [5]:
path = "./novel-corona-virus-2019-dataset/" # replace this with your path in gdrive

In [6]:
data = pd.read_csv(path + "covid_19_data.csv", header = 0, index_col = 0)
    
#     dropped_features = ["slug", "state_changed_at", "currency_symbol"]
#     data = data.drop(dropped_features, axis = 1)
#     combined_data = combined_data.append(data, ignore_index = True)

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
data.shape

(11614, 7)

In [9]:
data.columns.values

array(['ObservationDate', 'Province/State', 'Country/Region',
       'Last Update', 'Confirmed', 'Deaths', 'Recovered'], dtype=object)

In [10]:
data.head()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
SNo,,,,,,,
1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


### group by 'Country/Region' and 'ObservationDate' (ignore 'Province/State')

In [11]:
data_grouped = data.groupby(['ObservationDate','Country/Region'], as_index=False).sum()

In [12]:
data_grouped.shape

(6062, 5)

In [13]:
data_grouped.columns.values

array(['ObservationDate', 'Country/Region', 'Confirmed', 'Deaths',
       'Recovered'], dtype=object)

In [14]:
data_grouped

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,01/22/2020,Hong Kong,0.0,0.0,0.0
1,01/22/2020,Japan,2.0,0.0,0.0
2,01/22/2020,Macau,1.0,0.0,0.0
3,01/22/2020,Mainland China,547.0,17.0,28.0
4,01/22/2020,South Korea,1.0,0.0,0.0
...,...,...,...,...,...
6057,04/03/2020,Venezuela,153.0,7.0,52.0
6058,04/03/2020,Vietnam,237.0,0.0,85.0
6059,04/03/2020,West Bank and Gaza,194.0,1.0,21.0
6060,04/03/2020,Zambia,39.0,1.0,2.0


### sort by country, then date (within country )

In [15]:
data_grouped_country = data_grouped.sort_values(by=['Country/Region','ObservationDate'])

In [16]:
data_grouped_country.shape

(6062, 5)

In [17]:
data_grouped_country.columns.values

array(['ObservationDate', 'Country/Region', 'Confirmed', 'Deaths',
       'Recovered'], dtype=object)

In [18]:
data_grouped_country

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
1030,02/28/2020,Azerbaijan,1.0,0.0,0.0
1972,03/10/2020,"('St. Martin',)",2.0,0.0,0.0
848,02/24/2020,Afghanistan,1.0,0.0,0.0
886,02/25/2020,Afghanistan,1.0,0.0,0.0
928,02/26/2020,Afghanistan,1.0,0.0,0.0
...,...,...,...,...,...
2322,03/12/2020,occupied Palestinian territory,0.0,0.0,0.0
2594,03/14/2020,occupied Palestinian territory,0.0,0.0,0.0
2743,03/15/2020,occupied Palestinian territory,0.0,0.0,0.0
2901,03/16/2020,occupied Palestinian territory,0.0,0.0,0.0


### make non-cumulative columns for the dataframe named `[columnName]_diff`

In [19]:
data_grouped_country.reset_index(drop=True)

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,02/28/2020,Azerbaijan,1.0,0.0,0.0
1,03/10/2020,"('St. Martin',)",2.0,0.0,0.0
2,02/24/2020,Afghanistan,1.0,0.0,0.0
3,02/25/2020,Afghanistan,1.0,0.0,0.0
4,02/26/2020,Afghanistan,1.0,0.0,0.0
...,...,...,...,...,...
6057,03/12/2020,occupied Palestinian territory,0.0,0.0,0.0
6058,03/14/2020,occupied Palestinian territory,0.0,0.0,0.0
6059,03/15/2020,occupied Palestinian territory,0.0,0.0,0.0
6060,03/16/2020,occupied Palestinian territory,0.0,0.0,0.0


In [20]:
data_grouped_country.set_index(['Country/Region','ObservationDate'], inplace=True)
data_grouped_country.sort_index(inplace=True)
data_grouped_country['Confirmed_diff'] = np.nan
data_grouped_country['Deaths_diff'] = np.nan 
data_grouped_country['Recovered_diff'] = np.nan 

for idx in data_grouped_country.index.levels[0]:
    data_grouped_country.Confirmed_diff[idx] = data_grouped_country.Confirmed[idx].diff()
    data_grouped_country.Deaths_diff[idx] = data_grouped_country.Deaths[idx].diff()
    data_grouped_country.Recovered_diff[idx] = data_grouped_country.Recovered[idx].diff()

In [21]:
data_grouped_country.shape

(6062, 6)

In [22]:
data_grouped_country.columns.values

array(['Confirmed', 'Deaths', 'Recovered', 'Confirmed_diff',
       'Deaths_diff', 'Recovered_diff'], dtype=object)

In [23]:
data_grouped_country

Confirmed  Deaths  Recovered  \
Country/Region                 ObservationDate                                 
 Azerbaijan                    02/28/2020             1.0     0.0        0.0   
('St. Martin',)                03/10/2020             2.0     0.0        0.0   
Afghanistan                    02/24/2020             1.0     0.0        0.0   
                               02/25/2020             1.0     0.0        0.0   
                               02/26/2020             1.0     0.0        0.0   
...                                                   ...     ...        ...   
occupied Palestinian territory 03/12/2020             0.0     0.0        0.0   
                               03/14/2020             0.0     0.0        0.0   
                               03/15/2020             0.0     0.0        0.0   
                               03/16/2020             0.0     0.0        0.0   
                               03/17/2020             0.0     0.0        0.0   

                                                Confirmed_diff  Deaths_diff  \
Country/Region                 ObservationDate                                
 Azerbaijan                    02/28/2020                  NaN          NaN   
('St. Martin',)                03/10/2020                  NaN          NaN   
Afghanistan                    02/24/2020                  NaN          NaN   
                               02/25/2020                  0.0          0.0   
                               02/26/2020                  0.0          0.0   
...                                                        ...          ...   
occupied Palestinian territory 03/12/2020                  0.0          0.0   
                               03/14/2020                  0.0          0.0   
                               03/15/2020                  0.0          0.0   
                               03/16/2020                  0.0          0.0   
                               03/17/2020                  0.0          0.0   

                                                Recovered_diff  
Country/Region                 ObservationDate                  
 Azerbaijan                    02/28/2020                  NaN  
('St. Martin',)                03/10/2020                  NaN  
Afghanistan                    02/24/2020                  NaN  
                               02/25/2020                  0.0  
                               02/26/2020                  0.0  
...                                                        ...  
occupied Palestinian territory 03/12/2020                  0.0  
                               03/14/2020                  0.0  
                               03/15/2020                  0.0  
                               03/16/2020                  0.0  
                               03/17/2020                  0.0  

[6062 rows x 6 columns]

### drop non-numerical columns

In [24]:
# find and drop all non numerical columns -> leaving numerical data
non_numerical_columns = []
for column_name in data_grouped.columns.values:
    try:
        number = float(data[column_name].iloc[0])
    except:
        non_numerical_columns.append(column_name)
print (non_numerical_columns)

['ObservationDate', 'Country/Region']


In [25]:
# drop all the non numerical colums from above
numerical_data = data_grouped.drop(non_numerical_columns, axis = 1)

In [26]:
numerical_data.shape

(6062, 3)

In [27]:
numerical_data.head()

,Confirmed,Deaths,Recovered
0,0.0,0.0,0.0
1,2.0,0.0,0.0
2,1.0,0.0,0.0
3,547.0,17.0,28.0
4,1.0,0.0,0.0


### write the useful dataframes to a csv, save to `output` subfolder

In [28]:
data_grouped_country.to_csv(path + 'output/' + 'data_grouped_country.csv')
data_grouped.to_csv(path + 'output/' + 'data_grouped.csv')

Test Example for Edwin: Grab Lebanon Data and present neatly.

In [29]:
data_grouped_country.shape

(6062, 6)

In [30]:
data_grouped_country

Confirmed  Deaths  Recovered  \
Country/Region                 ObservationDate                                 
 Azerbaijan                    02/28/2020             1.0     0.0        0.0   
('St. Martin',)                03/10/2020             2.0     0.0        0.0   
Afghanistan                    02/24/2020             1.0     0.0        0.0   
                               02/25/2020             1.0     0.0        0.0   
                               02/26/2020             1.0     0.0        0.0   
...                                                   ...     ...        ...   
occupied Palestinian territory 03/12/2020             0.0     0.0        0.0   
                               03/14/2020             0.0     0.0        0.0   
                               03/15/2020             0.0     0.0        0.0   
                               03/16/2020             0.0     0.0        0.0   
                               03/17/2020             0.0     0.0        0.0   

                                                Confirmed_diff  Deaths_diff  \
Country/Region                 ObservationDate                                
 Azerbaijan                    02/28/2020                  NaN          NaN   
('St. Martin',)                03/10/2020                  NaN          NaN   
Afghanistan                    02/24/2020                  NaN          NaN   
                               02/25/2020                  0.0          0.0   
                               02/26/2020                  0.0          0.0   
...                                                        ...          ...   
occupied Palestinian territory 03/12/2020                  0.0          0.0   
                               03/14/2020                  0.0          0.0   
                               03/15/2020                  0.0          0.0   
                               03/16/2020                  0.0          0.0   
                               03/17/2020                  0.0          0.0   

                                                Recovered_diff  
Country/Region                 ObservationDate                  
 Azerbaijan                    02/28/2020                  NaN  
('St. Martin',)                03/10/2020                  NaN  
Afghanistan                    02/24/2020                  NaN  
                               02/25/2020                  0.0  
                               02/26/2020                  0.0  
...                                                        ...  
occupied Palestinian territory 03/12/2020                  0.0  
                               03/14/2020                  0.0  
                               03/15/2020                  0.0  
                               03/16/2020                  0.0  
                               03/17/2020                  0.0  

[6062 rows x 6 columns]

In [31]:
#countries = ["UK", "Kazakhstan", "Egypt", "Mexico"]
countries = ["India"]

In [32]:
country_data = data_grouped_country.loc["US"].reset_index()

In [33]:
#Logarithmic line graph

americalog = alt.Chart(data=country_data, width=400, height=300).transform_fold(
        ['Confirmed', 'Recovered', 'Deaths'],
        as_=['Key','Cases']
    ).transform_filter(
        (alt.datum['Recovered'] > 0) & alt.datum['Deaths'] > 0
    ).mark_line(point=True).encode(
        alt.X('ObservationDate', type='temporal', title="Observation Date"),
        alt.Y('Cases:Q', type='quantitative', title='Cases', scale=alt.Scale(type='log', base=10)),
        color='Key:N',
        tooltip=['ObservationDate:T','Key:N','Cases:Q']
    ).properties(
        title="US"
    )

americalog.save('americalog.html')

americalog

alt.Chart(...)

In [34]:
#Symlog graph.

symlog = alt.Chart(data=country_data, width=400, height=300).transform_fold(
        ['Confirmed', 'Recovered', 'Deaths'],
        as_=['Key','Cases']
    ).mark_line(point=True).encode(
        alt.X('ObservationDate', type='temporal', title="Observation Date"),
        alt.Y('Cases:Q', type='quantitative', title='Cases', scale=alt.Scale(type='symlog', base=10)),
        color='Key:N',
        tooltip=['ObservationDate:T','Key:N','Cases:Q']
    ).properties(
        title="US"
    )

symlog.save('symlogchart.html')

In [35]:
#Linear Bar Chart Graph

linear = alt.Chart(data=country_data, width=400, height=300).transform_fold(
        ['Confirmed', 'Recovered', 'Deaths'],
        as_=['Key','Cases']
    ).mark_bar().encode(
        alt.X('ObservationDate', type='temporal', title="Observation Date"),
        alt.Y('Cases:Q', type='quantitative', title='Cases'),
        color='Key:N',
        tooltip=['Key:N','Cases:Q']
    ).properties(
        title="US"
    )

linear.save('linearchart.html')

Combine data with HDI information.

In [36]:
hdi = pd.read_excel(path + "HDI_2018Data.xlsx", header = 5, index_col = 0, nrows=194, usecols=[0,1,2])

hdi.dropna(how='any',inplace=True)
hdi.rename(columns={'Country':'Country/Region'}, inplace=True)
data_grouped_country.reset_index(drop=False, inplace=True)

data_grouped_country = data_grouped_country.merge(hdi, on='Country/Region')
data_grouped_country.rename(columns={'(index value)': 'HDI'}, inplace=True)

In [37]:
hdibin = alt.Chart(data=data_grouped_country, width=400, height=300).mark_line(point=True).encode(
    alt.X('ObservationDate', type='temporal'),
    alt.Y('mean(Confirmed):Q', type='quantitative'),
    color='binned_agg:O',
    tooltip=['mean(Confirmed)']
).transform_bin(
    'binned_agg', 'HDI', bin=alt.Bin(maxbins=4)
).properties(
        title="Binned by HDI")

hdibin.save('hdiagg.html')

hdibin

alt.Chart(...)

Begin to add in new columns and information.

In [38]:
#Pour les reporteurs sans frontières

rsffop = pd.read_csv(path + "RSF_FreedomOfPressIndex_Data.csv", header=0, usecols=[1,3,7],decimal=',')
rsffop.rename(columns={'EN_country':'Country/Region'}, inplace=True)

data_grouped_country = data_grouped_country.merge(rsffop, on='Country/Region')



In [39]:
data_grouped_country.rename(columns={'Score 2019':'FOP','Rank2019':'FOP Rank'}, inplace=True)

In [56]:
fopbin = alt.Chart(data=data_grouped_country, width=400, height=300).mark_line(point=True).encode(
    alt.X('ObservationDate', type='temporal', title='Observation Date'),
    alt.Y('mean(Confirmed):Q', type='quantitative', title='Mean of Confirmed Cases in Bucket'),
    color='FOP_Bin:O',
    tooltip=['mean(Confirmed)','FOP_Bin:O']
).transform_bin(
    'FOP_Bin', 'FOP', bin=alt.Bin(maxbins=4)
).properties(
        title="Confirmed Cases of SARS-CoV-2, Bucketed By Press Freedoms")

fopbin.save('fopagg.html')

fopbin

alt.Chart(...)

In [ ]:
health = pd.read_excel(path + "HDI_2018Data.xlsx", sheet_name='Table 8',header = 5, nrows=194, usecols=[1,30])

health.dropna(how='any',inplace=True)
health.rename(columns={'Country':'Country/Region', '(% of GDP)':'Health Expenditure '}, inplace=True)

data_grouped_country = data_grouped_country.merge(health, on='Country/Region')

In [ ]:
data_grouped_country

In [ ]:
healthbin = alt.Chart(data=data_grouped_country, width=400, height=300).mark_line(point=True).encode(
    alt.X('ObservationDate', type='temporal'),
    alt.Y('mean(Confirmed):Q', type='quantitative'),
    color='binned_agg:O',
    tooltip=['mean(Confirmed)']
).transform_bin(
    'binned_agg', 'Health Expenditure', bin=alt.Bin(maxbins=4)
).properties(
        title="Binned by Health")

healthbin.save('healthagg.html')

healthbin